In [ ]:
import numpy as np
import laspy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
from pathlib import Path

## Load LAS Files from data_red folder

In [ ]:
# Set the data directory
data_dir = Path('data_red')

# List all .las files
las_files = list(data_dir.glob('*.las'))
print(f"Found {len(las_files)} LAS files:")
for f in las_files:
    print(f"  - {f.name}")

## Load and Inspect a Single LAS File

In [ ]:
# Load the first LAS file
las_file_path = las_files[0]
print(f"Loading: {las_file_path}")

# Read the LAS file
las_data = laspy.read(las_file_path)

# Print basic information
print(f"\nNumber of points: {len(las_data.points)}")
print(f"Point format: {las_data.point_format}")
print(f"\nAvailable dimensions:")
for dim in las_data.point_format.dimension_names:
    print(f"  - {dim}")

## Extract Point Cloud Data

In [ ]:
# Extract coordinates
x = las_data.x
y = las_data.y
z = las_data.z

print(f"X range: [{x.min():.2f}, {x.max():.2f}]")
print(f"Y range: [{y.min():.2f}, {y.max():.2f}]")
print(f"Z range: [{z.min():.2f}, {z.max():.2f}]")

# Check for classification (labels)
if hasattr(las_data, 'classification'):
    classification = las_data.classification
    unique_classes = np.unique(classification)
    print(f"\nUnique classification labels: {unique_classes}")
    print(f"Number of unique classes: {len(unique_classes)}")
    for cls in unique_classes:
        count = np.sum(classification == cls)
        print(f"  Class {cls}: {count} points ({count/len(classification)*100:.2f}%)")
else:
    print("\nNo classification information found")

# Check for RGB colors
has_rgb = False
if hasattr(las_data, 'red') and hasattr(las_data, 'green') and hasattr(las_data, 'blue'):
    has_rgb = True
    red = las_data.red
    green = las_data.green
    blue = las_data.blue
    print(f"\nRGB information available")
else:
    print(f"\nNo RGB information available")

# Check for intensity
if hasattr(las_data, 'intensity'):
    intensity = las_data.intensity
    print(f"Intensity range: [{intensity.min()}, {intensity.max()}]")

## 3D Visualization - Full Point Cloud

In [ ]:
# Subsample for faster visualization (take every N-th point)
subsample_factor = max(1, len(x) // 50000)  # Limit to ~50k points
x_sub = x[::subsample_factor]
y_sub = y[::subsample_factor]
z_sub = z[::subsample_factor]

print(f"Visualizing {len(x_sub)} points (subsampled by factor {subsample_factor})")

# Create 3D plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot points colored by height (z-coordinate)
scatter = ax.scatter(x_sub, y_sub, z_sub, c=z_sub, cmap='viridis', s=1, alpha=0.6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z (Height)')
ax.set_title(f'Point Cloud: {las_file_path.name}')
plt.colorbar(scatter, label='Height', shrink=0.5)
plt.tight_layout()
plt.show()

## 3D Visualization - Colored by Classification

In [ ]:
if hasattr(las_data, 'classification'):
    classification_sub = classification[::subsample_factor]
    
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    scatter = ax.scatter(x_sub, y_sub, z_sub, c=classification_sub, 
                        cmap='tab10', s=1, alpha=0.6)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z (Height)')
    ax.set_title(f'Point Cloud Colored by Classification: {las_file_path.name}')
    plt.colorbar(scatter, label='Class', shrink=0.5)
    plt.tight_layout()
    plt.show()
else:
    print("Classification not available for this file")

## Top-Down View (2D Projection)

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(x_sub, y_sub, c=z_sub, cmap='viridis', s=0.5, alpha=0.6)
plt.xlabel('X')
plt.ylabel('Y')
plt.title(f'Top-Down View (colored by height): {las_file_path.name}')
plt.colorbar(label='Height (Z)')
plt.axis('equal')
plt.tight_layout()
plt.show()

## Height Distribution

In [ ]:
plt.figure(figsize=(12, 4))
plt.hist(z, bins=100, alpha=0.7, edgecolor='black')
plt.xlabel('Height (Z)')
plt.ylabel('Number of Points')
plt.title(f'Height Distribution: {las_file_path.name}')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Convert to Numpy Array for Deep Learning

In [ ]:
# Create a numpy array with XYZ coordinates
points = np.vstack((x, y, z)).T
print(f"Point cloud shape: {points.shape}")
print(f"Data type: {points.dtype}")

# Normalize coordinates (important for deep learning)
points_normalized = points - points.mean(axis=0)
max_distance = np.max(np.linalg.norm(points_normalized, axis=1))
points_normalized = points_normalized / max_distance

print(f"\nNormalized point cloud shape: {points_normalized.shape}")
print(f"Normalized range: [{points_normalized.min():.3f}, {points_normalized.max():.3f}]")

# If we have additional features, we can concatenate them
features_list = [points]

if hasattr(las_data, 'intensity'):
    # Normalize intensity
    intensity_norm = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-8)
    features_list.append(intensity_norm.reshape(-1, 1))
    print("Added intensity feature")

if has_rgb:
    # Normalize RGB
    rgb = np.vstack((red, green, blue)).T
    rgb_norm = rgb / 65535.0  # LAS RGB values are typically 16-bit
    features_list.append(rgb_norm)
    print("Added RGB features")

# Concatenate all features
point_features = np.hstack(features_list)
print(f"\nFinal feature array shape: {point_features.shape}")
print(f"Features per point: {point_features.shape[1]}")

## Load and Compare Multiple Files

In [ ]:
# Load all files and show basic statistics
print("Summary of all LAS files in data_red:\n")
print(f"{'Filename':<25} {'Points':<12} {'Classes':<10}")
print("-" * 50)

for las_file in las_files:
    las = laspy.read(las_file)
    n_points = len(las.points)
    
    if hasattr(las, 'classification'):
        n_classes = len(np.unique(las.classification))
    else:
        n_classes = "N/A"
    
    print(f"{las_file.name:<25} {n_points:<12} {str(n_classes):<10}")

## Summary

This notebook demonstrated:
1. Loading .las files using laspy
2. Extracting XYZ coordinates and other attributes (classification, intensity, RGB)
3. 3D visualization of point clouds
4. Data preprocessing for deep learning (normalization)
5. Creating feature arrays combining multiple attributes

Next steps:
- Use these preprocessed point clouds for training PointNet models
- Implement data augmentation techniques
- Create train/test splits